In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

In [3]:
model_checkpoints = "gogamza/kobart-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
# train_category_path = "data/train_category.csv"
# val_category_path = "data/val_category.csv"

# with open(train_category_path, encoding="utf-8") as f:
#             train_category = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

# with open(val_category_path, encoding="utf-8") as f:
#             val_category = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

In [5]:
train_textfile_path = "data/train_text.csv"
train_summaryfile_path = "data/train_summary.csv"

with open(train_textfile_path, encoding="utf-8") as f:
            train_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(train_summaryfile_path, encoding="utf-8") as f:
            train_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]          

In [6]:
val_textfile_path = "data/val_text.csv"
val_summaryfile_path = "data/val_summary.csv"

with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(val_summaryfile_path, encoding="utf-8") as f:
            val_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]          

In [7]:
del train_textlines[0]
del val_textlines[0]
# del train_category[0]
# del val_category[0]
del val_sumlines[0]
del train_sumlines[0]

In [8]:
train_df = pd.DataFrame(zip(train_textlines, train_sumlines), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_textlines, val_sumlines), columns=['Text', 'Summary'])

In [9]:
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [10]:
train_df.head()

,Text,Summary
0,그럼 날짜는 가격 큰 변동 없으면 6 28-7 13로 확정할까 우리 비행포함 15일...,비행기 표 가격에 대해 이야기하며 특가 이벤트를 기다리고 있다
1,kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마스크말고 부직포 마...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다
2,아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애 그러...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕칫솔 또 사려나 싶...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다
4,잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보 빼이보릿 되버렸네...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다


In [11]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)

In [12]:
print(train_data)
print(val_data)

Dataset({
    features: ['Text', 'Summary'],
    num_rows: 279992
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 35004
})


In [13]:
max_input = 128
max_target = 32
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [14]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [15]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [16]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i]), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Summary'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [17]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/280 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [18]:
# # set special tokens
#from transformers import EncoderDecoderConfig
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 32 # 256은 쿠다 메모리 오류 생김
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 5
model.config.length_penalty = 2.0
model.config.num_beams = 5
#model.config.suppress_tokens = [23782, 14338, 22554, 234]


In [19]:
model.config

BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
 

In [20]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)
   

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/stopword_test",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=3,

    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [23]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
   # callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
trainer.train()

/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 279992
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 87500
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.743400
1000,3.769600
1500,3.639400


Saving model checkpoint to checkpoint/stopword_test/checkpoint-500
Configuration saved in checkpoint/stopword_test/checkpoint-500/config.json
Model weights saved in checkpoint/stopword_test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in checkpoint/stopword_test/checkpoint-500/tokenizer_config.json
Special tokens file saved in checkpoint/stopword_test/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [checkpoint/stopword_test/checkpoint-16500] due to args.save_total_limit
Saving model checkpoint to checkpoint/stopword_test/checkpoint-1000
Configuration saved in checkpoint/stopword_test/checkpoint-1000/config.json
Model weights saved in checkpoint/stopword_test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in checkpoint/stopword_test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in checkpoint/stopword_test/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [checkpoint/stopword_test/checkpoint-17000] due to a

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=5,no_repeat_ngram_size=5,
                            attention_mask=attention_mask, top_p=0.92,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            eos_token_id=tokenizer.eos_token_id,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
summaries_before_tuning = generate_summary(test_data, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_data, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

In [ ]:
class Queue:
    # 초기 설정, 스택으로 사용할 리스트 선언
    def __init__(self):
        self.queue = []
 
    # In 함수 / insert 함수 List의 Insert 함수 이용
    def insert(self, data):
        self.queue.insert(0, data)
 
    # Out 함수 / Remove 함수, List의 pop함수 이용
    def remove(self):
        if len(self.queue) <=0 :
            return("No Data in Stack")
        else :
            return self.queue.pop()
summaries_after_tuning = Queue()

In [ ]:
val_textfile_path = "data/val_text.csv"
val_summaryfile_path = "data/val_summary.csv"

with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(val_summaryfile_path, encoding="utf-8") as f:
            val_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]      
del val_textlines[0]
del val_sumlines[0]
test_df = pd.DataFrame(zip(val_textlines, val_sumlines), columns=['Text', 'Summary'])
test_df.reset_index(inplace=True, drop=True)
test_samples = Dataset.from_pandas(test_df)

In [ ]:
for test_sample in tqdm(test_samples):
    summaries_after_tuning.insert(generate_summary(test_sample, model)[1])

In [ ]:
rouge.get_scores(summaries_after_tuning, test_samples["Summary"], avg=True)

In [ ]:
for i in range(0, len(summaries_after_tuning), 1000):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  